In [1]:
import IJulia
import Base64

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = "L2hvbWUvbmljbGFzL0RvY3VtZW50cy93ZWJzaXRlL05pY2xSaWNoLmdpdGh1Yi5pby9ibG9nX3Bvc3Rz"
  if !isempty(run_path)
    run_path = String(Base64.base64decode(run_path))
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
# implement a structure for the problem
mutable struct kdv_problem{F <: AbstractFloat}
    alpha::F            # α in the equation
    beta::F             # β in the equation
    gamma::F            # γ in the equation

    delta_x::F          # spatial resolution
    delta_t::F          # time resolution

    interval_length::F  # interval length of the base

    T::F                # end time 

    u::Matrix{F}        # array for the solution
end

# Outer constructor for automated sizing the array and initializing it with zeros
"""
    kdv_problem(alpha, beta, gamma, delta_x, T, interval_length, initial_values)

Set up Korteweg-de Vries (KdV) equation with initial values and suitable parameters.
Compute the time step automatically such that it is a stable scheme.

# Arguments
- `alpha::F`: Parameter for the equation.
- `beta::F`: Parameter for the equation.
- `gamma::F`: Parameter for the equation.
- `delta_x::F`: Spatial resolution.
- `T::F`: End time for the simulation.
- `interval_length::F`: Length of the spatial interval.
- `initial_values::Vector{F}`: Initial values for the simulation.
- `delta_t`=Nothing: Time resolution, if `Nothing` then it calculates it using the criterion given by Taha1984.

# Returns
- `kdv_problem{F}`: An object representing the KdV problem with the specified parameters.

# Example
alpha = 1.0
beta = 2.0
gamma = 3.0
delta_x = 0.01
T = 5.0
interval_length = 1.0
initial_values = [sin(2 * π * x) for x in 0:delta_x:interval_length]

kdv = kdv_problem(alpha, beta, gamma, delta_x, T, interval_length, initial_values)

"""
function kdv_problem(alpha::F, beta::F, gamma::F, delta_x::F, T::F, interval_length::F, initial_values::Vector{F}, delta_t::Union{F, Nothing} = Nothing) where F <: AbstractFloat
    if T <= 0.0
        error("Negative end time")
    elseif interval_length <= 0.0
        error("Negative interval length")
    elseif delta_x <= 0.0
        error("Negative spatial resolution")
    end

    if isnothing(delta_t)
        delta_t = T
        numerical_stable = false

        # compute the time step
        while !numerical_stable
            delta_t = 0.5 * delta_t
            numerical_stable = delta_t / delta_x * abs((- 2 * maximum(initial_values) + 1 / (delta_x^2))) <= 2 / (3 * sqrt(3))
        end
    end

    # compute the dimensions of the matrix for storing values
    N_x = ceil(Int64, interval_length / delta_x)
    N_t = ceil(Int64, T / delta_t) + 1 # plus one for the implicit scheme at the beginning
    u = zeros(F, N_x, N_t)
    u[:, 2] .= initial_values # set the initial values
    
    return kdv_problem{F}(alpha, beta, gamma, delta_x, delta_t, interval_length, T, u)
end

In [3]:
"""
    solve!(P::kdv_problem)

Solve the Korteweg-de Vries (KdV) equation numerically using an explicit finite difference scheme.
The Zabusky-Kruskal scheme with periodic boundary conditions is used.

# Arguments
- `P::kdv_problem`: KdV Equation with initial values.


# Returns
- `kdv_problem{F}`: An object representing the KdV problem with the specified parameters.

# Example
alpha = 1.0
beta = 2.0
gamma = 3.0
delta_x = 0.01
T = 5.0
interval_length = 1.0
initial_values = [sin(2 * π * x) for x in 0:delta_x:interval_length]

kdv = kdv_problem(alpha, beta, gamma, delta_x, T, interval_length, initial_values)
solve!(kdv)
"""
function solve!(P::kdv_problem)
    N_x, N_t = size(P.u)

    # use an implicit scheme to solve for the time - Δt, since two time steps 
    # are necessary for the Zabusky-Kruskal scheme

    for i in 0:N_x-1
        # Calculate the indices according to the boundary conditions
        in2 = mod(i + 2, N_x) + 1   # two indices to the right (next index)
        in1 = mod(i + 1, N_x) + 1   # one index to the right (next index)
        ip1 = mod(i - 1, N_x) + 1      # one index before, (to the left)
        ip2 = mod(i - 2, N_x) + 1      # two indices before, (to the left)
        k = i + 1

        dispersion = P.beta * ( -P.u[ip2, 2] + 2 * P.u[ip1, 2] - 2 * P.u[in1, 2] + P.u[in2, 2]) / (2 * P.delta_x^3)
        advection = (P.gamma / 3) * (P.u[ip1, 2] + P.u[k, 2] + P.u[in1, 2]) * ((P.u[in1, 2] - P.u[ip1, 2]) / (2 * P.delta_x))
        P.u[k, 1] = P.u[k, 2] + (P.delta_t / P.alpha) * (dispersion + advection)
        
    end

    # now apply the ZK-scheme since the initial values and the values at -Δt
    # are known

    for j in 3:N_t
        for i in 0:N_x-1
        # Calculate the indices according to the boundary conditions
            in2 = mod(i + 2, N_x) + 1   # two indices to the right (next index)
            in1 = mod(i + 1, N_x) + 1   # one index to the right (next index)
            ip1 = mod(i - 1, N_x) + 1      # one index before, (to the left)
            ip2 = mod(i - 2, N_x) + 1      # two indices before, (to the left)
            k = i + 1

            dispersion = P.beta * ( -P.u[ip2, j-1] + 2 * P.u[ip1, j-1] - 2 * P.u[in1, j-1] + P.u[in2, j-1]) / (2 * P.delta_x^3)
            advection = (P.gamma / 3) * (P.u[ip1, j-1] + P.u[k, j-1] + P.u[in1, j-1]) * ((P.u[in1, j-1] - P.u[ip1, j-1]) / (2 * P.delta_x))
            P.u[k, j] = P.u[k, j - 2] - (2 * P.delta_t / P.alpha) * (dispersion + advection)
        end
    end
end

In [4]:
"""
test_ZK_scheme()

Test the Zk-scheme for the solution

u(x,t) = 0.5 * sech(0.5 * (x - t) - 4.0)^2

for the KdV equation
    0 = 1.0 * u_t + 1.0 * u_xxx + 6.0 * u_x * u 
"""
function test_ZK_scheme()
    σ = 1.0
    start_point  = 0.0
    end_point = 20
    interval_length = end_point - start_point
    end_time = 0.2
    exact_sol(x,t) = 0.5 * σ * sech(sqrt(σ) * 0.5 * (x - σ*t) -4.0)^2
    inital_values(x) = exact_sol(x, 0.0)

    for k in 0:6
        # Set up the problem
        dx = 0.5^k
        x = collect(range(start_point, end_point, step= dx))
        initial_values = inital_values.(x)
        P = kdv_problem(1.0, 1.0, 6.0, dx, end_time, interval_length, initial_values)
        
        # solve the Problem
        solve!(P)
        
        # Compute the error
        dt = P.delta_t
        approximate_solution = P.u[:, end]
        exact_solution = exact_sol.(x, end_time)
        err_vec = abs.(exact_solution - approximate_solution)
        # compute the error norms
        err_inf_norm = maximum(err_vec)
        err_1_norm = trapezoidal_rule(x, err_vec)
        err_2_norm = sqrt(trapezoidal_rule(x, err_vec .* err_vec))
        println("+"^80)
        println("k = $k")
        @printf "dx = %.3e\n" dx
        @printf "dt = %.3e\n" dt
        @printf "error_inf_norm = %.3e \n" err_inf_norm
        @printf "error_1_norm = %.3e \n" err_1_norm
        @printf "error_2_norm = %.3e \n" err_2_norm
    end
    
end

"""
    trapezoidal_rule(x, y)

Compute the numerical approximation of the definite integral using the trapezoidal rule.

# Arguments
- `x::Vector`: Vector of sample points representing the independent variable.
- `y::Vector`: Vector of function values corresponding to each sample point.

# Output
- Returns the numerical approximation of the definite integral using the trapezoidal rule.

# Examples
x_values = [0.0, 0.1, 0.2, 0.3, 0.4]
y_values = [f(x) for x in x_values]  # Replace f(x) with your function
result = trapezoidal_rule(x_values, y_values)
println("Trapezoidal Rule Integral: ", result)
"""
function trapezoidal_rule(x, y)
    h = diff(x)
    return sum((y[1:end-1] + y[2:end]) .* h) * 0.5
end

In [5]:
function show_conservation()
    # set up the KdV Equation and solve it
    σ = 1.0
    start_point  = 0.0
    end_point = 20
    interval_length = end_point - start_point
    end_time = 0.2
    exact_sol(x,t) = 0.5 * σ * sech(sqrt(σ) * 0.5 * (x - σ*t) -4.0)^2
    inital_values(x) = exact_sol(x, 0.0)

    dx = 0.5^4
    x = collect(range(start_point, end_point, step= dx))
    initial_values = inital_values.(x)
    P = kdv_problem(1.0, 1.0, 6.0, dx, end_time, interval_length, initial_values)
    
    # solve the Problem
    solve!(P)

    # extract the values and compute the conserved values
    u = P.u
    u2 = u .* u

    c1 = mapslices(a -> trapezoidal_rule(x, a), u, dims =1)
    c2 = mapslices(a -> trapezoidal_rule(x, a), u2, dims = 1)
    diff_c1 = maximum(c1) - minimum(c1)
    diff_c2 = maximum(c2) - minimum(c2)

    @printf "First two conserved quantities of the KdV-Equation:"
    @printf "c1 ≈ %.8e \n" sum(c1) / length(c1)
    @printf "Maximal difference observed: %.3e\n" diff_c1
    @printf "\n"
    @printf "c1 ≈ %.8e \n" sum(c2) / length(c2)
    @printf "Maximal difference observed: %.3e\n" diff_c2
    
    
end

show_conservation()